### Required library

In [1]:
import os
from dotenv import load_dotenv
from coinbase.rest import RESTClient
from json import dumps
from datetime import datetime, timedelta
import pandas as pd
import uuid

##### Link : https://docs.cdp.coinbase.com/advanced-trade/docs/sdk-rest-client-trade/

### 1) Provide API key

In [2]:
load_dotenv()
api_key = os.getenv("api_key")
api_secret = os.getenv("api_secret")

# Create Instance
client = RESTClient(api_key=api_key, api_secret=api_secret)

### 2) Get account information

In [3]:
accounts = client.get_accounts()
# print(dumps(accounts, indent=2))

### 3) Get specific account information

In [4]:
account_uuid = os.getenv("account_uuid")
account_details = client.get_account(account_uuid)
# print(dumps(account_details, indent=2))

### 4) Get Time

In [5]:
time_dict = client.get_unix_time()
iso_time = time_dict["iso"]
epoch_seconds = int(time_dict["epochSeconds"])
epoch_millis = int(time_dict["epochMillis"])

print(f"Time_dict : {time_dict}")
print(f"ISO Time : {iso_time}")
print(f"Epoch Seconds : {epoch_seconds}")
print(f"Epoch Millis : {epoch_millis}")

Time_dict : {'iso': '2024-09-22T02:06:59Z', 'epochSeconds': '1726970819', 'epochMillis': '1726970819980'}
ISO Time : 2024-09-22T02:06:59Z
Epoch Seconds : 1726970819
Epoch Millis : 1726970819980


### 5) Retrieve Fees Charged

In [6]:
product_type = "SPOT"
contract_expiry_type = "UNKNOW_CONTRACT"
summary = client.get_transaction_summary()
print(type(summary))
print(summary)

<class 'dict'>
{'total_volume': 1.2579989, 'total_fees': 0.0150959872, 'fee_tier': {'pricing_tier': 'Advanced 1', 'usd_from': '0', 'usd_to': '1000', 'taker_fee_rate': '0.012', 'maker_fee_rate': '0.006', 'aop_from': '', 'aop_to': ''}, 'margin_rate': None, 'goods_and_services_tax': None, 'advanced_trade_only_volume': 1.2579989, 'advanced_trade_only_fees': 0.0150959872, 'coinbase_pro_volume': 0, 'coinbase_pro_fees': 0, 'total_balance': '8.84', 'has_promo_fee': False}


### 6) Get Market Data

In [7]:
product_id = "BTC-USD"

# 86400 = 24 x 60 x 60
end = datetime.now()
start = end - timedelta(hours=24)

end_unix = int(end.timestamp())
start_unix = int(start.timestamp())

# ONE_MINUTE, FIVE_MINUTE, FIFTEEN_MINUTE, THIRTY_MINUTE, ONE_HOUR, TWO_HOUR, SIX_HOUR, ONE_DATE
granularity = "ONE_HOUR"

data = client.get_candles(product_id, start_unix, end_unix, granularity)
candle_data = data["candles"]

# unit = (D,s,ms,us,ns)
df_candles = pd.DataFrame(candle_data)
df_candles['start'] = pd.to_datetime(df_candles['start'], unit='s')

df_candles.head()
# print(end_unix)
# print(start_unix)

/var/folders/2l/k8sqwr_j3gs338sv_rft9kjm0000gn/T/ipykernel_3228/2130603460.py:18: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_candles['start'] = pd.to_datetime(df_candles['start'], unit='s')


,start,low,high,open,close,volume
0,2024-09-22 01:59:28,63001.01,63111.55,63050.01,63011.69,24.78385521
1,2024-09-22 00:59:44,63022.51,63307.82,63239.87,63050.16,70.35648229
2,2024-09-22 00:00:00,63123.12,63467.99,63362.74,63243.02,143.15137339
3,2024-09-21 23:00:16,63163.3,63575.63,63233.02,63362.74,177.4608383
4,2024-09-21 22:00:32,63058.63,63249.61,63098.76,63233.05,58.67386718


### 7) Get Trade Data

In [8]:
product_id = "BTC-USD"
limit = "50"
data = client.get_market_trades(product_id, limit)
trade_data = data["trades"]

df_trade = pd.DataFrame(trade_data)
df_trade.head()


,trade_id,product_id,price,size,time,side,bid,ask
0,694666959,BTC-USD,63012.75,0.006322,2024-09-22T02:07:19.021713Z,SELL,,
1,694666958,BTC-USD,63007.19,0.00299276,2024-09-22T02:07:18.689708Z,BUY,,
2,694666957,BTC-USD,63007.96,0.00077,2024-09-22T02:07:18.689708Z,BUY,,
3,694666956,BTC-USD,63007.97,0.00043202,2024-09-22T02:07:18.689708Z,BUY,,
4,694666955,BTC-USD,63010.51,0.00077,2024-09-22T02:07:18.689708Z,BUY,,


### 8) Place an order

- uuid 모듈은 Universally Unique Identifier(전 세계적으로 고유한 식별자)를 생성
- UUID는 16바이트(128비트)의 값으로 표현
- uuid4() 함수는 랜덤 UUID를 생성
- hex는 32자리의 16진수 문자열을 반환

In [9]:
client_order_id = str(uuid.uuid4().hex)
client_order_id

'53775a71abc64bb69f9db007699a001e'

In [17]:
product_id = "BTC-USD"
quote_size = "0.62" # $
order_data = client.market_order_buy(client_order_id=client_order_id, product_id=product_id, quote_size=quote_size)
print(dumps(order_data, indent=2))

{
  "success": true,
  "success_response": {
    "order_id": "e51d9871-cecf-4b51-962a-23a1237edc68",
    "product_id": "BTC-USD",
    "side": "BUY",
    "client_order_id": "1560dd6435e54a779dea04cf5dd532ae",
    "attached_order_id": ""
  },
  "order_configuration": {
    "market_market_ioc": {
      "quote_size": "0.62"
    }
  }
}


#### Get order detail

In [18]:
order_id = order_data.get("success_response", {}).get("order_id")
order_details = client.get_order(order_id)
print(order_details)

{'order': {'order_id': 'e51d9871-cecf-4b51-962a-23a1237edc68', 'product_id': 'BTC-USD', 'user_id': '241ed7c1-eaa4-5044-89d8-7a64de1ae161', 'order_configuration': {'market_market_ioc': {'base_size': '0.00001'}}, 'side': 'SELL', 'client_order_id': '1560dd6435e54a779dea04cf5dd532ae', 'status': 'FILLED', 'time_in_force': 'IMMEDIATE_OR_CANCEL', 'created_time': '2024-09-20T00:04:44.626527Z', 'completion_percentage': '100.00', 'filled_size': '0.00001', 'average_filled_price': '62933.38', 'fee': '', 'number_of_fills': '1', 'filled_value': '0.6293338', 'pending_cancel': False, 'size_in_quote': False, 'total_fees': '0.0075520056', 'size_inclusive_of_fees': False, 'total_value_after_fees': '0.6217817944', 'trigger_status': 'INVALID_ORDER_TYPE', 'order_type': 'MARKET', 'reject_reason': 'REJECT_REASON_UNSPECIFIED', 'settled': True, 'product_type': 'SPOT', 'reject_message': '', 'cancel_message': '', 'order_placement_source': 'RETAIL_ADVANCED', 'outstanding_hold_amount': '0', 'is_liquidation': False,

In [19]:
order_id = order_data
print(order_id)

{'success': True, 'success_response': {'order_id': 'e51d9871-cecf-4b51-962a-23a1237edc68', 'product_id': 'BTC-USD', 'side': 'BUY', 'client_order_id': '1560dd6435e54a779dea04cf5dd532ae', 'attached_order_id': ''}, 'order_configuration': {'market_market_ioc': {'quote_size': '0.62'}}}


In [25]:
order_id = order_data.get("success_response",{})
print(order_id)

{'order_id': 'e51d9871-cecf-4b51-962a-23a1237edc68', 'product_id': 'BTC-USD', 'side': 'BUY', 'client_order_id': '1560dd6435e54a779dea04cf5dd532ae', 'attached_order_id': ''}


In [27]:
order_id = order_data.get("success_response", {}).get("order_id")
print(order_id)

e51d9871-cecf-4b51-962a-23a1237edc68


In [30]:
order_details = client.get_order(order_id)
print(order_details)

{'order': {'order_id': 'e51d9871-cecf-4b51-962a-23a1237edc68', 'product_id': 'BTC-USD', 'user_id': '241ed7c1-eaa4-5044-89d8-7a64de1ae161', 'order_configuration': {'market_market_ioc': {'base_size': '0.00001'}}, 'side': 'SELL', 'client_order_id': '1560dd6435e54a779dea04cf5dd532ae', 'status': 'FILLED', 'time_in_force': 'IMMEDIATE_OR_CANCEL', 'created_time': '2024-09-20T00:04:44.626527Z', 'completion_percentage': '100.00', 'filled_size': '0.00001', 'average_filled_price': '62933.38', 'fee': '', 'number_of_fills': '1', 'filled_value': '0.6293338', 'pending_cancel': False, 'size_in_quote': False, 'total_fees': '0.0075520056', 'size_inclusive_of_fees': False, 'total_value_after_fees': '0.6217817944', 'trigger_status': 'INVALID_ORDER_TYPE', 'order_type': 'MARKET', 'reject_reason': 'REJECT_REASON_UNSPECIFIED', 'settled': True, 'product_type': 'SPOT', 'reject_message': '', 'cancel_message': '', 'order_placement_source': 'RETAIL_ADVANCED', 'outstanding_hold_amount': '0', 'is_liquidation': False,

### 9) Place market Order SELL

In [31]:
client_order_id = str(uuid.uuid4().hex)
product_id = "BTC-USD"
base_size = "0.00001" #BTC 
order_data = client.market_order_sell(client_order_id=client_order_id, product_id=product_id, base_size=base_size)
print(dumps(order_data, indent=2))

{
  "success": true,
  "success_response": {
    "order_id": "3e2c7444-288f-4219-af92-140c97518c2f",
    "product_id": "BTC-USD",
    "side": "SELL",
    "client_order_id": "24ed2d26da2c4e9cabef8b46b3ac31a2",
    "attached_order_id": ""
  },
  "order_configuration": {
    "market_market_ioc": {
      "base_size": "0.00001"
    }
  }
}


#### Get order detail

In [32]:
order_id = order_data.get("success_response", {}).get("order_id")
order_details = client.get_order(order_id)
print(order_details)

{'order': {'order_id': '3e2c7444-288f-4219-af92-140c97518c2f', 'product_id': 'BTC-USD', 'user_id': '241ed7c1-eaa4-5044-89d8-7a64de1ae161', 'order_configuration': {'market_market_ioc': {'base_size': '0.00001'}}, 'side': 'SELL', 'client_order_id': '24ed2d26da2c4e9cabef8b46b3ac31a2', 'status': 'FILLED', 'time_in_force': 'IMMEDIATE_OR_CANCEL', 'created_time': '2024-09-20T00:12:46.419592Z', 'completion_percentage': '100.00', 'filled_size': '0.00001', 'average_filled_price': '62866.51', 'fee': '', 'number_of_fills': '1', 'filled_value': '0.6286651', 'pending_cancel': False, 'size_in_quote': False, 'total_fees': '0.0075439812', 'size_inclusive_of_fees': False, 'total_value_after_fees': '0.6211211188', 'trigger_status': 'INVALID_ORDER_TYPE', 'order_type': 'MARKET', 'reject_reason': 'REJECT_REASON_UNSPECIFIED', 'settled': True, 'product_type': 'SPOT', 'reject_message': '', 'cancel_message': '', 'order_placement_source': 'RETAIL_ADVANCED', 'outstanding_hold_amount': '0', 'is_liquidation': False,

In [33]:
order_details = client.get_order(order_id)
print(order_details)

{'order': {'order_id': '3e2c7444-288f-4219-af92-140c97518c2f', 'product_id': 'BTC-USD', 'user_id': '241ed7c1-eaa4-5044-89d8-7a64de1ae161', 'order_configuration': {'market_market_ioc': {'base_size': '0.00001'}}, 'side': 'SELL', 'client_order_id': '24ed2d26da2c4e9cabef8b46b3ac31a2', 'status': 'FILLED', 'time_in_force': 'IMMEDIATE_OR_CANCEL', 'created_time': '2024-09-20T00:12:46.419592Z', 'completion_percentage': '100.00', 'filled_size': '0.00001', 'average_filled_price': '62866.51', 'fee': '', 'number_of_fills': '1', 'filled_value': '0.6286651', 'pending_cancel': False, 'size_in_quote': False, 'total_fees': '0.0075439812', 'size_inclusive_of_fees': False, 'total_value_after_fees': '0.6211211188', 'trigger_status': 'INVALID_ORDER_TYPE', 'order_type': 'MARKET', 'reject_reason': 'REJECT_REASON_UNSPECIFIED', 'settled': True, 'product_type': 'SPOT', 'reject_message': '', 'cancel_message': '', 'order_placement_source': 'RETAIL_ADVANCED', 'outstanding_hold_amount': '0', 'is_liquidation': False,

### Get list of order

In [36]:
list_orders = client.list_orders()
orders_data = list_orders.get('orders', {})
df = pd.json_normalize(orders_data)
df

,order_id,product_id,user_id,side,client_order_id,status,time_in_force,created_time,completion_percentage,filled_size,...,is_liquidation,last_fill_time,edit_history,leverage,margin_type,retail_portfolio_id,originating_order_id,attached_order_id,attached_order_configuration,order_configuration.market_market_ioc.base_size
0,3e2c7444-288f-4219-af92-140c97518c2f,BTC-USD,241ed7c1-eaa4-5044-89d8-7a64de1ae161,SELL,24ed2d26da2c4e9cabef8b46b3ac31a2,FILLED,IMMEDIATE_OR_CANCEL,2024-09-20T00:12:46.419592Z,100.00,0.00001,...,False,2024-09-20T00:12:46.482272Z,[],,UNKNOWN_MARGIN_TYPE,241ed7c1-eaa4-5044-89d8-7a64de1ae161,,,None,0.00001
1,e51d9871-cecf-4b51-962a-23a1237edc68,BTC-USD,241ed7c1-eaa4-5044-89d8-7a64de1ae161,SELL,1560dd6435e54a779dea04cf5dd532ae,FILLED,IMMEDIATE_OR_CANCEL,2024-09-20T00:04:44.626527Z,100.00,0.00001,...,False,2024-09-20T00:04:44.690981Z,[],,UNKNOWN_MARGIN_TYPE,241ed7c1-eaa4-5044-89d8-7a64de1ae161,,,None,0.00001
